In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from statsmodels.tsa.arima_model import ARIMA

E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
#Population values
population = {
    "Chiapas_2016-2017.csv": 5217908,
    "Colima_2016-2017.csv": 711235,
    "Guerrero_2016-2017.csv": 3533251,
    "Hidalgo_2016-2017.csv": 2858359,
    "NuevoLeon_2016-2017.csv": 5119504,
    "Oaxaca_2016-2017.csv": 3967889,
    "QuintanaRoo_2016-2017.csv": 1501562,
    "Tabasco_2016-2017.csv" : 2395272,
    "Veracruz_2016-2017.csv" : 8112505,
    "Yucatan_2016-2017.csv" : 2097175,
    
    "casanare_2016-2017.csv" : 356438,
    "cordoba_2016-2017.csv" : 1709603,
    "cundinamarca_2016-2017.csv" : 2680041,
    "huila_2016-2017.csv" : 1154804,
    "meta_2016-2017.csv" : 961292,
    "santander_2016-2017.csv" : 2061095,
    "santander_norte_2016-2017.csv" : 1355723,
    "tolima_2016-2017.csv" : 1408274,
    "valle_cauca_2016-2017.csv" : 4613377,
    
    "Alagoas_2016-2017.csv": 3375823,
    "Bahia_2016-2017.csv": 15344447,
    "Ceara_2016-2017.csv": 9020460,
    "Goias_2016-2017.csv": 6778772,
    "Maranhao_2016-2017.csv": 7000229,
    "MatoGrosso_2016-2017.csv": 3344544,
    "MinasGerais_2016-2017.csv": 21119536,
    "Para_2016-2017.csv": 8366628,
    "RioDeJaneiro_2016-2017.csv": 16718956,
    "SaoPaulo_2016-2017.csv": 45094866,
}

In [40]:
ArimaParameters = {
    "Brazil" : [1, 0, 0],
    "Colombia" : [3, 0, 0],
    "Mexico" : [4, 0, 1],
}

In [41]:
test_index = 26
end_index = 104
for country in ["Brazil", "Colombia", "Mexico"]:
    
    print("Country - {}".format(country))
    p, d, q = ArimaParameters[country]
    
    folder = "../../data/{}/processed_data".format(country)
    files = os.listdir(folder)
    
    for file in files:
        
        print("File - {}".format(file))
        if(not os.path.exists("{}/{}".format(country, file))):
            os.mkdir("{}/{}".format(country, file))
        
        dataset = pd.read_csv("{}/{}".format(folder, file))
        dataset["Searches"] /= 100
        state = file
        dataset["Cases"] *= (100000/population[state]) # Transform to incidence per 100,000 habitants  
        
        predictions_df = pd.DataFrame()
        predictions_df["Observed Cases"] = dataset["Cases"][test_index:].values * (population[state] / 100000)
        
        predictionsArr= []
        for i in range(test_index, end_index):
            
            model= ARIMA(dataset["Cases"][:i].values,
             order=(p,d,q), freq="W")
            
            results = model.fit(trend="nc")
            
            predictions = results.predict(
                start=i,
                end=i
            )
            predictionsArr.append(predictions[0]) #CHECK
        predictions_df["Predicted Cases"] = predictionsArr
        predictions_df["Predicted Cases"] *= (population[state] / 100000)
        
        predictions_df["error"] = predictions_df["Predicted Cases"]  - predictions_df["Observed Cases"]
        predictions_df.to_csv("{}/{}/Arimax-{}".format(country, file, file))
        
        #Save images
        plt.clf()

        cols = ['Observed Cases', 'Predicted Cases']
        colors = ['#2962FF', '#FF9800']

        predictions_df[cols].plot(
            figsize=(10, 10),
            grid=True,
            color=colors
        )

        plt.title("Arimax Model\n{}".format(file.replace(".csv", "").replace("_2016-"," ")))
        ax = plt.gca()
        ax.set_facecolor((0.9, 0.9, 0.9, 0.7))
        plt.xlabel("Date")
        plt.ylabel("ZIKV Cases")
        plt.grid(linestyle='dashed', linewidth=1.5)
        fig = plt.gcf()
        fig.savefig("{}/{}/Arimax-{}.png".format(country, file, file.replace(".csv", "")))
        plt.close("all")

Country - Brazil
File - Bahia_2016-2017.csv


E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
E:\Aplicaciones\Anaconda\lib\site-packages\statsmodels\base\model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


File - MatoGrosso_2016-2017.csv
File - MinasGerais_2016-2017.csv
File - RioDeJaneiro_2016-2017.csv
File - SaoPaulo_2016-2017.csv
Country - Colombia
File - huila_2016-2017.csv
File - santander_2016-2017.csv
File - santander_norte_2016-2017.csv
File - tolima_2016-2017.csv
File - valle_cauca_2016-2017.csv
Country - Mexico
File - Chiapas_2016-2017.csv


ValueError: The computed initial AR coefficients are not stationary
You should induce stationarity, choose a different model order, or you can
pass your own start_params.